In [37]:
import os
import sys
import importlib

# this can disapear once plex is a pip package
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
import plex.sdk
importlib.reload(plex.sdk)

os.environ["PLEX_ACCESS_TOKEN"] = "mellon"
os.environ["PLEX_ENV"] = "stage"
os.environ["DOCKER_DEFAULT_PLATFORM"] = "linux/x86_64/v8"

In [42]:
import csv
import os
import json

def create_pdbind_io_dict(csv_path, size=float('inf')):
    io_graph = []

    i = 0
    with open(csv_path, 'r') as csvfile:
        csvreader = csv.DictReader(csvfile)

        for row in csvreader:
            protein_path = os.path.join("/home/ubuntu/", row['protein_path'])
            ligand_path = os.path.join("/home/ubuntu/", row['ligand_description'])
            
            if not os.path.exists(protein_path) or not os.path.exists(ligand_path):
                print(f"Skipping row {row['complex_name']} due to missing file(s).")
                continue

            io_subgraph = {
                "tool": "tools/equibind.json",
                "inputs": {
                    "protein": {
                        "class": "File",
                        "filepath": protein_path
                    },
                    "small_molecule": {
                        "class": "File",
                        "filepath": ligand_path
                    }
                },
                "outputs": {
                    "best_docked_small_molecule": {
                        "class": "File",
                        "filepath": ""
                    },
                    "protein": {
                        "class": "File",
                        "filepath": ""
                    }
                },
                "state": "created",
                "errMsg": ""
            },
            {
                "tool": "tools/oddt.json",
                "inputs": {
                    "protein": {
                        "class": "File",
                        "filepath": f"${{{i}[protein]}}"
                    },
                    "small_molecule": {
                        "class": "File",
                        "filepath": f"${{{i}[best_docked_small_molecule]}}"
                    }
                },
                "outputs": {
                    "best_docked_small_molecule": {
                        "class": "File",
                        "filepath": ""
                    },
                    "protein": {
                        "class": "File",
                        "filepath": ""
                    }
                },
                "state": "created",
                "errMsg": "" 
            },
            {
                "outputs": {
                    "scored_small_molecule": {
                        "class": "File",
                        "filepath": ""
                    },
                    "scores": {
                        "class": "File",
                        "filepath": ""
                    }
                },
                "tool": "tools/openbabel/rmsd-openbabel.json",
                "inputs": {
                    "reference_structure": {
                        "class": "File",
                        "filepath": f"${{{i}[protein]}}"
                    },
                    "comparison_structure": {
                        "class": "File",
                        "filepath": f"${{{i}[best_docked_small_molecule]}}"
                    }
                },
                "state": "created",
                "errMsg": ""
            }

            i += 1
            if i >= size:
                break
            io_graph.append(io_subgraph)
    return io_graph

# Example usage

csv_path = '/home/ubuntu/datasets/diffdock_testdata.csv'
io_graph = create_pdbind_io_dict(csv_path, 2)


FileNotFoundError: [Errno 2] No such file or directory: '/home/ubuntu/datasets/diffdock_testdata.csv'

In [41]:
from plex.sdk import run_plex

run_plex(io_graph, local=True, verbose=True, concurrency=2)

Plex version (v0.6.1) up to date.
BACALHAU_API_HOST not set, using default host
toolPath 
Running IPWL io path
Created working directory:  /Users/mcmenemy/code/plex/a921e615-a956-4e0c-9db8-dd27dd78363b
Reading IO Entries from:  /var/folders/yh/vwqzmpsd55xchsjvhxvbnqvh0000gn/T/tmp7hwy5x5t/io_data.json
Initialized IO file at:  /Users/mcmenemy/code/plex/a921e615-a956-4e0c-9db8-dd27dd78363b/io.json
Processing IO Entries
/Users/mcmenemy/code/plex/a921e615-a956-4e0c-9db8-dd27dd78363b
/Users/mcmenemy/code/plex/a921e615-a956-4e0c-9db8-dd27dd78363b/io.json
Starting to process IO entry 0 
Starting to process IO entry 2 
IO Subgraph at 2 is still waiting on inputs to complete 
Success processing IO entry 2 
Starting to process IO entry 1 
IO Subgraph at 1 is still waiting on inputs to complete 
Success processing IO entry 1 
Generated docker cmd: docker run  -v /Users/mcmenemy/code/plex/a921e615-a956-4e0c-9db8-dd27dd78363b/entry-0/inputs:/inputs -v /Users/mcmenemy/code/plex/a921e615-a956-4e0c-9db